In [1]:
import sys
import json
from shapely.geometry import shape
from utils.deafrica_datahandling import load_ard
from utils.deafrica_plotting import rgb
import datacube
from datacube.utils.geometry import Geometry

In [2]:
# Initialize datacube instance
dc = datacube.Datacube(app='Rainfall_HN')

# Load the GeoJSON data for the specific province
with open('./vietnam-provinces-iso.json') as f:  # Replace with your file path
    geojson_data = json.load(f)

# # Load the GeoJSON data for the specific province
# with open('./vietnam-districts-iso.json') as f:  # Replace with your file path
#     geojson_data = json.load(f)


In [3]:

# Extract geometry for Hà Nội province
hanoi_geometry = shape(geojson_data['features'][51]['geometry'])
# hanoi_geometry = shape(geojson_data['features'][466]['geometry'])
print(hanoi_geometry);


MULTIPOLYGON (((105.8297 21.3751, 105.8307 21.3739, 105.8336 21.375, 105.8353 21.3776, 105.8365 21.3782, 105.845 21.3786, 105.8451 21.3783, 105.8464 21.3763, 105.8469 21.3758, 105.8475 21.3755, 105.8493 21.3758, 105.8525 21.3771, 105.8536 21.3774, 105.8546 21.3774, 105.856 21.3766, 105.8569 21.3752, 105.8574 21.374, 105.8576 21.369, 105.8575 21.3678, 105.857 21.3672, 105.8564 21.3666, 105.8546 21.3659, 105.8523 21.3642, 105.851 21.3622, 105.8504 21.3602, 105.8498 21.3574, 105.8506 21.3512, 105.8511 21.3493, 105.8513 21.3489, 105.8533 21.3447, 105.8553 21.341, 105.8567 21.339, 105.8631 21.331, 105.865 21.3293, 105.866 21.329, 105.8668 21.3287, 105.8759 21.3278, 105.8809 21.3273, 105.8846 21.3268, 105.8877 21.3264, 105.8899 21.3264, 105.8918 21.3269, 105.8926 21.327, 105.895 21.3276, 105.8961 21.3273, 105.8967 21.3268, 105.8972 21.3258, 105.8974 21.324, 105.8962 21.3165, 105.8954 21.3118, 105.8955 21.3095, 105.8971 21.3067, 105.9009 21.3043, 105.9041 21.3028, 105.9066 21.3031, 105.9115 2

In [4]:

# Create a query with the geometry
query = {
    'time': ('2024-10-01',),  # Adjust time range as needed
    'geopolygon': Geometry(hanoi_geometry, crs="EPSG:4326"),
    'output_crs': 'EPSG:4326',
    'resolution': (-0.1, 0.1)
}


In [5]:
# Load data using the geometry query
ds = dc.load(product='imerg_e_10KM_daily', dask_chunks={}, **query)

In [6]:
# Calculate average rainfall in this area (mm/day)
average_rainfall_all_area = ds['Precipitation'].mean(dim=['time', 'latitude', 'longitude'])

print(average_rainfall_all_area.values)

126.12345679012346


In [7]:
# Calculate daily total rainfall by summing over latitude and longitude for each day (mm/day*(lat *lon))
total_rainfall_all_area = ds['Precipitation'].sum(dim=['latitude', 'longitude'])

print(total_rainfall_all_area.values)

[10216]


In [8]:
# Calculate minimum rainfall in this area for the day (mm/day)
min_rainfall_all_area = ds['Precipitation'].min(dim=['latitude', 'longitude'])
print(min_rainfall_all_area.values)

# Calculate maximum rainfall in this area for the day (mm/day)
max_rainfall_all_area = ds['Precipitation'].max(dim=['latitude', 'longitude'])
print(max_rainfall_all_area.values)


[15]
[580]


In [9]:
# Create a query with the geometry and date range
query_october = {
    'time': ('2024-10-01', '2024-10-31'),  # Adjust time range as needed
    'geopolygon': Geometry(hanoi_geometry, crs="EPSG:4326"),
    'output_crs': 'EPSG:4326',
    'resolution': (-0.1, 0.1)
}

In [10]:
# Load data using the geometry query
ds_october = dc.load(product='imerg_e_10KM_daily', dask_chunks={}, **query_october)

In [11]:
# Calculate total rainfall for the entire area in October (summed over time, latitude, and longitude)
total_rainfall_october = ds_october['Precipitation'].sum(dim=['time', 'latitude', 'longitude'])
print(total_rainfall_october.values)


80454


In [12]:
# Calculate average daily rainfall in October by taking the mean over the time, latitude, and longitude)
average_daily_rainfall_october = ds_october['Precipitation'].mean(dim=['time', 'latitude', 'longitude'])
print(average_daily_rainfall_october.values)

32.04062126642772


In [13]:
def load_data(time_range=None):
    # Tạo query cho toàn bộ dữ liệu mà không giới hạn theo thời gian
    query = {
        'geopolygon': Geometry(hanoi_geometry, crs="EPSG:4326"),
        'output_crs': 'EPSG:4326',
        'resolution': (-1, 1)
    }
    
    # Chỉ thêm thời gian vào query nếu time_range được cung cấp
    if time_range:
        query['time'] = time_range

    # Sử dụng dask_chunks để chia nhỏ dữ liệu
    ds = dc.load(product='imerg_e_10KM_daily', dask_chunks={'time': 10, 'latitude': 50, 'longitude': 50}, **query)
    
    # Persist dữ liệu vào bộ nhớ
    ds['Precipitation'] = ds['Precipitation'].persist()
    return ds


In [14]:
def calculate_total_rainfall(time_range, freq):
    ds = load_data(time_range)
    
    # Resample theo 'time' và tính tổng sau khi gộp theo 'latitude' và 'longitude'
    total_rainfall = ds['Precipitation'].resample(time=freq).sum(dim='time').sum(dim=['latitude', 'longitude'])
    
    # Trả về giá trị tổng sau khi gộp
    return total_rainfall


In [15]:
def calculate_average_rainfall(time_range, freq):
    ds = load_data(time_range)
    
    # Sử dụng Dask để thực hiện tính trung bình với hiệu suất cao
    average_rainfall = ds['Precipitation'].resample(time=freq).mean(dim='time').mean(dim=['latitude', 'longitude'])
    
    # Tính toán trên bộ nhớ nếu có thể
#     average_rainfall = average_rainfall.compute()
    
    return average_rainfall


In [16]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [17]:
# Tính tổng và trung bình lượng mưa theo ngày trong tháng 10
day_total_rainfall = calculate_total_rainfall(('2024-10-01', '2024-10-31'), 'D')

print(day_total_rainfall.values)


[10216  2826    58     0    41    40    97     0     0    19     0    18
    33    15   142 11416   323     0    15  3808   901  5232   184    31
    48   351 14427 17473 10492  1052  1196]


In [18]:
# # Tính tổng và trung bình lượng mưa theo ngày trong tháng 10
# day_average_rainfall = calculate_average_rainfall(('2024-10-01', '2024-10-31'), 'D')

# print(day_average_rainfall.values)

In [20]:
# Tính tổng lượng mưa hàng tuần trên toàn bộ dữ liệu
week_total_rainfall = calculate_total_rainfall(None, 'W')
print(week_total_rainfall.values)


[   537   2988   1227   3434  18164   9113   8190  11557   9039   2857
   7303  43599  29253   1515   2421  22996  11795  14389  12181  14866
  66896  19252  77296  61375  60526  36232  33111  24744  54280 153532
  88287  27589  52274 180781  24772 121641 127232 113587  27540  39822
    167  15719  21174  30318  15307     99]


In [21]:
# Tính tổng lượng mưa hàng tuần trên toàn bộ dữ liệu
month_total_rainfall = calculate_total_rainfall(None, 'M')
print(month_total_rainfall.values)


[ 21040  39917  86304  38753 125725 237262 316119 320832 419060  80454
  15511]


In [22]:
# Tính tổng lượng mưa hàng tuần trên toàn bộ dữ liệu
quarter_total_rainfall = calculate_total_rainfall(None, 'Q')
print(quarter_total_rainfall.values)


[ 147261  401740 1056011   95965]


In [23]:
# Tính tổng lượng mưa hàng tuần trên toàn bộ dữ liệu
year_total_rainfall = calculate_total_rainfall(None, 'Y')
print(year_total_rainfall.values)


[1700977]
